In [16]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torchsummary import summary
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.__version__
print(device)

cuda


## Utils and Input & Output Process

In [96]:
# ============================== Utils =========================================
def print_train_time(start: float, end: float, device: torch.device = None):
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

def plot_loss_accuracy(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['categorical_accuracy'])
    if ('val_categorical_accuracy' in history.history.keys()):
        plt.plot(history.history['val_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    if ('val_loss' in history.history.keys()):
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def adBits(x, x2, y, seqsize=100):
   for i in range(x.shape[0]):
      #print(' for each train sample  ... ', i)
      loop = seqsize
      j = 0
      for loop in range(seqsize):     
          #print(' for each node ... ', j)
          adBits_1 = int(x[i][2*loop])
          #Get the next LONG since each node now has 2 LONGs for adj edges
          adBits_2 = int(x[i][2*loop+1])
          # Uncomment below if you want invalid nodes to not feed to NN before any valid nodes
          #if (adBits_1==0 and adBits_1==0 and loop!=seqsize-1):              
          #    y[i][j] = y[i][loop+1] 
          #    continue
          if (adBits_1):
              #print adBits
              for k in range(0,64):
                  bit = adBits_1 & 1
                  #print ( ' adBits&1 = ',  bit )
                  # Uncomment below if you want new nodes to only have adjacency to earlier nodes
                  if ( bit == 1):
                  #if ( bit == 1 and k<j):
                      x2[i][j][k] = 1
                      #print x2[i-1][j-1][k-1]
                  else:
                      x2[i][j][k] = 0
                      #print x2[i-1][j-1][k-1]
                  adBits_1 >>= 1
              x2[i][j][j] = 1 

          if (adBits_2):   
              #print adBits
              for k in range(64,100):
                  bit = adBits_2 & 1
                  #print ( ' adBits&1 = ',  bit )
                  if ( bit == 1):
                  #if ( bit == 1 and k<j):
                      x2[i][j][k] = 1
                      #print x2[i-1][j-1][k-1]
                  else:
                      x2[i][j][k] = 0
                      #print x2[i-1][j-1][k-1]
                  adBits_2 >>= 1
              x2[i][j][j] = 1
          j = j+1
   return x2, y

def updateLabelBits(x, y, seqsize=100):
    for i in range(x.shape[0]):
        #print(' for each train sample  ... ', i)
        label_dict = {}
        label_num = 1
        for j in range(seqsize):
            if (x[i][j][j] == 0):
                y[i][j] = 0
            else:
                if y[i][j] in label_dict:
                    y[i][j] = label_dict[y[i][j]]
                else:
                    label_dict[y[i][j]] = label_num
                    y[i][j] = label_num
                    label_num = label_num + 1
    return y

# ============================== Post Process ==================================

csv_rows = []

def post_process (x2_pred, predicted, seqsize=100):
    #Calculate the number of edges which will require correction
    invCols = 0
    edges = 0
    for i in range(x2_pred.shape[0]):
        for j in range(seqsize): # row
            for k in range(j): # col
                adj = x2_pred[i][j][k]
                if (adj == 1):
                    edges += 1
                    if (np.argmax(predicted[i][j]) == np.argmax(predicted[i][k])):
                        invCols += 1
                        
    print('Total No of edges ', edges)
    print('# of edges with invalid coloring ', invCols)
    print('Total percentage of edges with invalid colors ', invCols/edges)

def post_process_chromatic (x2_pred, predicted, seqsize=100):  
    colors_list_list = []
    for i in range(x2_pred.shape[0]):
        colors_list = []
        for j in range(seqsize):
            # Valid nodes will have below set to 1 so check the color 
            # assignment of those nodes only
            if (x2_pred[i][j][j] != 0):
                colors_list.append(np.argmax(predicted[i][j]))
        print('Colors list of graph ', i, ' is  \n', colors_list)
        chromatic_number = len(set(colors_list))
        print('Chromatic number of graph ', i, ' is  ', chromatic_number)
        colors_list_list.append(colors_list)
    return colors_list_list

def create_csv_rows (graph_name, colors_list_list_before_correction, 
                     colors_list_list_after_correction):    
    for i in range(len(colors_list_list_before_correction)):
        row = [graph_name, i, len(set(colors_list_list_before_correction[i])), 
               len(set(colors_list_list_after_correction[i]))]        
        csv_rows.append(row)

def post_process_correction (x2_pred, predicted, colors_list_list, seqsize=100): 
  totInvCols = 0
  totEdges = 0

  for i in range(x2_pred.shape[0]):
      #maxcol = max(xpredicted[i])
      maxcol = max(colors_list_list[i])
      #print(maxcol)
      #mcol = maxcol[0]
      maxorigcol = maxcol
      mcolnew = maxcol
      #print('Maxcol = ',maxcol[0])
      #print(' ... FOR SAMPLE  ... ', i)
      invCols = 0
      edges = 0;
      newCol = 500

      for j in range(seqsize):
          #print(' ... ... FOR EACH NODE ... ...', j)
          for k in range(j):
              #print(' ... ... ... for each adjacency  ... ... ...', k)
              adj = x2_pred[i][j][k]
              #There is an edge
              if ( adj == 1 ):
                  edges += 1
                  if ( np.argmax(predicted[i][j]) == np.argmax(predicted[i][k]) ):                   
                      col_j = np.argmax(predicted[i][j])
                      col_k = np.argmax(predicted[i][k])
                      invCols += 1

                      #Check whether we can give one of the existing colors
                      foundfinalcol = 0
                      for  y in range(1,maxcol+1):
                          #print('Check for COLOR NO ... ', y)
                          if ( foundfinalcol == 1 ) :
                              #print('FOUND COLOR ALREADY  ... leave the loop')
                              break

                          foundcol = 0
                          #Check the adjacent nodes of j
                          #for  z in range(j):
                          for z in range(seqsize):
                              if j!=z:
                                  if  (   ((x2_pred[i][j][z] == 1) and (np.argmax(predicted[i][z]) == y))
                                      or  ((x2_pred[i][z][j] == 1) and (np.argmax(predicted[i][z]) == y))
                                      ):
                                      #print('[1] Adjacent node ... from ',j, '-->', z, 'color = ',xpredicted[i][z][0] )
                                      foundcol = 1
                                      #print('[1] Found Color ', y, ' for node ', z, 'from node ', j )
                                      break

                          #Finished checking the adjacent nodes of j
                          #Color y is not used by any of j's neighbours
                          #print('[1] Finished Checking the adjacent node of ... ',j,' ... foundcol = ',foundcol)
                          if ( foundcol == 0 ) :
                              #assign any prediction > 1
                              predicted[i][j][y] = 2
                              #print('[1] Reuse color ', y, ' for node ', j)
                              foundfinalcol = 1

                          else :
                              foundcol = 0                                                            
                              #Check the adjacent nodes of k
                              for z in range(seqsize):
                                  if k!=z:
                                      if  (   ((x2_pred[i][k][z] == 1) and (np.argmax(predicted[i][z]) == y))
                                          or  ((x2_pred[i][z][k] == 1) and (np.argmax(predicted[i][z]) == y))
                                          ):
                                          #print('[1] Adjacent node ... from ',j, '-->', z, 'color = ',xpredicted[i][z][0] )
                                          foundcol = 1
                                          #print('[1] Found Color ', y, ' for node ', z, 'from node ', j )
                                          break
                              #Color y is not used by any of k's neighbours
                              if ( foundcol == 0 ) :
                                  #assign any prediction > 1
                                  predicted[i][k][y] = 2
                                  #print('[2] Reuse color ', y, ' for node ', k )
                                  foundfinalcol = 1

                      # Could not color using an existing color
                      # Get a new color from 500 onwards OR use from the new 500 color number series
                      if ( foundfinalcol == 0 ) :
                           #newCol += 1
                           mcolnew += 1
                           #assign any prediction > 1
                           predicted[i][k][mcolnew] = 2
                           maxcol +=1
                           #print('Use new color ', mcolnew, ' for node ', k)

  return predicted



## Data Process

In [65]:
seq_size = 100
seq = pd.read_csv("train_data.csv",header=None,low_memory=False)
columns = seq.columns.tolist()

#get the adj edges
adj_edge = np.array(seq[columns[1:2*seq_size+1]])
#get the colors
data_color = np.array(seq[columns[2*seq_size+1:3*seq_size+1]])
print("shape 1: ", adj_edge.shape, data_color.shape)

X = np.zeros((adj_edge.shape[0], data_color.shape[1], seq_size))
print("shape 2: ", X.shape)

X, Y = adBits(adj_edge, X, data_color)
Y = updateLabelBits(X, Y)


print("shape 3: ", X.shape, Y.shape)


shape 1:  (8158, 200) (8158, 100)
shape 2:  (8158, 100, 100)
shape 3:  (8158, 100, 100) (8158, 100)


In [66]:
Y = np.eye(101, dtype='float32')[Y]
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)
X.shape, Y.shape

(torch.Size([8158, 100, 100]), torch.Size([8158, 100, 101]))

In [67]:
x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2, 
                                                    random_state=42)
x_train, x_test= x_train.to(device), x_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([6526, 100, 100]),
 torch.Size([6526, 100, 101]),
 torch.Size([1632, 100, 100]),
 torch.Size([1632, 100, 101]))

In [30]:
seqsize = 100

eseq1 = pd.read_csv("train_data.csv",header=None,low_memory=False)

eseq  = eseq1

columns = eseq.columns.tolist()
#2 entries for each node's adjacency 
edata = eseq[columns[1:3*seqsize+1]]
#get the adj edges
edataadj = eseq[columns[1:2*seqsize+1]]
#get the colors
edatacols = eseq[columns[2*seqsize+1:3*seqsize+1]]
#chromatic number
ncols = eseq[columns[0:1]]

#Now make x_train a 2-dim array
#As we have read the adjacency row as bits
#and we now want to make them into features
#x_train and x2_train have different shapes as 2 LONGs are present in x_train
#So use y_train.shape[1] instead of x_train.shape[1]
#x2_train = np.zeros((x_train.shape[0],y_train.shape[1],128))
from keras.utils import to_categorical

x_train, y_train = np.array(edataadj), np.array(edatacols)
x2_train = np.zeros((x_train.shape[0], y_train.shape[1], seqsize))
x2_train, y_train = adBits(x_train, x2_train, y_train)
y_train = updateLabelBits(x2_train, y_train)
y2_train = to_categorical(y_train, num_classes=101)
print(x2_train.shape)
print(y2_train.shape)


2023-12-07 11:26:41.668696: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 11:26:41.902653: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 11:26:41.902807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 11:26:41.941682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 11:26:42.023480: I tensorflow/core/platform/cpu_feature_guar

(8158, 100, 100)
(8158, 100, 101)


In [57]:
# y2_train[0][0:3]
# y_train.shape, y_train[38], y2_train[38][3]
x2_train[38, 0:4, 0:6], x_train[38, 0:6], y_train[38, 0:6], y2_train[38, 0:4, 0:6]

(array([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0., 0.]]),
 array([ 6.,  0., 13.,  0., 11.,  0.]),
 array([1, 2, 3, 1, 0, 0]),
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=float32))

In [118]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, seq_size=100, device=device):
        seq = pd.read_csv(img_dir, header=None, low_memory=False)
        columns = seq.columns.tolist()
        #get the adj edges
        adj_edge = np.array(seq[columns[1:2*seq_size+1]])
        #get the colors
        data_color = np.array(seq[columns[2*seq_size+1:3*seq_size+1]])
        self.X = np.zeros((adj_edge.shape[0], data_color.shape[1], seq_size))
        self.X, self.Y = adBits(adj_edge, self.X, data_color)
        self.Y = updateLabelBits(self.X, self.Y)
        self.Y = np.eye(101, dtype='float32')[self.Y]
        self.X = torch.tensor(self.X, dtype=torch.float32).to(device)
        self.Y = torch.tensor(self.Y, dtype=torch.float32).to(device)
        print("X shape: ", self.X.shape, ", Y shape: ", self.Y.shape)
        # print(self.Y[38, 0:4, 0:6])
        # X shape:  torch.Size([8158, 100, 100]) , Y shape:  torch.Size([8158, 100, 101])
        # tensor([[0., 1., 0., 0., 0., 0.],
        #         [0., 0., 1., 0., 0., 0.],
        #         [0., 0., 0., 1., 0., 0.],
        #         [0., 1., 0., 0., 0., 0.]], device='cuda:0')

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        ig = self.X[idx] # interference graph
        label = self.Y[idx]
        return ig, label        

In [119]:
BATCH_SIZE = 32
full_dataset = CustomDataset(
    img_dir="train_data.csv",
)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_data, test_data = torch.utils.data.random_split(full_dataset, 
                                                      [train_size, test_size])

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")



X shape:  torch.Size([8158, 100, 100]) , Y shape:  torch.Size([8158, 100, 101])
Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7f56a59662b0>, <torch.utils.data.dataloader.DataLoader object at 0x7f55c06c8dc0>)
Length of train dataloader: 204 batches of 32
Length of test dataloader: 51 batches of 32


## Build Model

In [142]:
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)
        y = self.module(x_reshape)
        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)
        return y


class DLRegAlloc(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm_1 = nn.LSTM(input_size=100, hidden_size=512, 
                              batch_first=True, bidirectional=True)
        self.lstm_2 = nn.LSTM(input_size=1024, hidden_size=256, 
                              batch_first=True,bidirectional=True)
        self.lstm_3 = nn.LSTM(input_size=512, hidden_size=128, 
                              batch_first=True,bidirectional=True)
        # self.lstm_1 = nn.LSTM(input_size=512, hidden_size=256, bidirectional=True)
        # self.lstm_2 = nn.LSTM(input_size=256, hidden_size=128, bidirectional=True)
        # self.lstm_3 = nn.LSTM(input_size=128, hidden_size=64, bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(in_features=256, out_features=101)
        self.softmax = nn.Softmax(dim=1)
        self.time_distributed = TimeDistributed(self.softmax)
        
    def forward(self, x):
        # print("----: ", x.shape)
        x, _ = self.lstm_1(x)
        # print("lstm1 shape: ", x.shape)
        x = self.dropout(x)

        x, _ = self.lstm_2(x)
        # print("lstm2 shape: ", x.shape)
        x = self.dropout(x)

        x, _ = self.lstm_3(x)
        # print("lstm3 shape: ", x.shape)
        x = self.dropout(x)

        x = self.fc(x)
        # print("fc shape: ", x.shape)

        x = self.time_distributed(x)
        # print("time_distributed shape: ", x.shape)

        return x


model = DLRegAlloc().to(device)


In [135]:
test_x, test_y = next(iter(train_dataloader))
test_pred = model(test_x)
test_x.shape, test_y.shape, test_pred.shape, test_x[0, 0:6, 0:6], \
    test_y[0, 0:6, 0:6], test_pred.shape, test_pred[0, 0:6, 0:6]

(torch.Size([32, 100, 100]),
 torch.Size([32, 100, 101]),
 torch.Size([32, 100, 101]),
 tensor([[1., 0., 1., 0., 0., 0.],
         [0., 1., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0., 0.],
         [0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.]], device='cuda:0'),
 tensor([[0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.]], device='cuda:0'),
 torch.Size([32, 100, 101]),
 tensor([[0.0106, 0.0106, 0.0095, 0.0094, 0.0100, 0.0102],
         [0.0106, 0.0106, 0.0095, 0.0093, 0.0099, 0.0102],
         [0.0105, 0.0104, 0.0096, 0.0095, 0.0097, 0.0102],
         [0.0106, 0.0105, 0.0095, 0.0093, 0.0097, 0.0101],
         [0.0108, 0.0105, 0.0094, 0.0091, 0.0097, 0.0102],
         [0.0108, 0.0106, 0.0095, 0.0094, 0.0099, 0.0101]], device='cuda:0',
        grad_fn=<SliceBackward0>))

In [123]:
test_pred.argmax(dim=2)

tensor([[54, 54, 54,  ..., 64, 64, 64],
        [60, 54, 54,  ..., 64, 64, 64],
        [54, 54, 64,  ..., 64, 64, 64],
        ...,
        [54, 54, 54,  ..., 64, 54, 64],
        [54, 54, 54,  ..., 64, 64, 64],
        [54, 54, 64,  ..., 64, 64, 64]], device='cuda:0')

In [124]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
from helper_functions import accuracy_fn

In [155]:
def train_step(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device,
               train_loss_values=None,
               train_acc_values=None,
               tracking=False
               ):
    train_loss, train_acc = 0, 0
    # model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        model.train()
        # +++++++
        # y_pred shape:  torch.Size([32, 100, 101])
        # y shape:  torch.Size([32, 100, 101])
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=y_pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    if tracking:
        train_loss_values.append(train_loss.clone().detach().cpu().numpy())
        train_acc_values.append(train_acc)
    print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")

def test_step(model: torch.nn.Module, 
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module, 
              accuracy_fn,
              device: torch.device = device,
              test_loss_values=None,
              test_acc_values=None,
              tracking=False):
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X_test, y_test in data_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_pred = model(X_test)
            test_loss += loss_fn(test_pred, y_test)
            test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred)
        test_loss /= len(data_loader)
        test_acc /= len(data_loader) 
        if tracking:
            test_loss_values.append(test_loss.clone().detach().cpu().numpy())
            test_acc_values.append(test_acc)
        print(f"Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")


        

## Train

In [156]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from tqdm.auto import tqdm
from timeit import default_timer as timer

train_time_start_model_2 = timer()

train_loss_values = []
train_acc_values = []
test_loss_values = []
test_acc_values = []
epoch_count = []

epochs = 100
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n------")
    tracking = False
    if epoch % 10 == 0:
        tracking = True
        epoch_count.append(epoch)
    train_step(model=model,
               data_loader=train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_fn,
               device=device,
               train_loss_values=train_loss_values,
               train_acc_values=train_acc_values,
               tracking=tracking)
    test_step(data_loader=test_dataloader,
            model=model,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device,
            test_loss_values=test_loss_values,
            test_acc_values=test_acc_values,
            tracking=tracking)

train_time_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_time_start_model_2,
                                           end=train_time_end_model_2,
                                           device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0
------
__+++_+++
<class 'torch.Tensor'>
<class 'float'>
Train loss: 4.55956 | Train acc: 0.00%
Test loss: 4.55956, Test acc: 0.00%

Epoch: 1
------


RuntimeError: cudnn RNN backward can only be called in training mode

In [ ]:

MODEL_SAVE_PATH = "torch_version.pth"

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 


## Plot loss & accuracy

In [ ]:
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend();